## 패키지 영구 설치하기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import pandas as pd
from pandas import DataFrame  as df
import konlpy
import nltk
from nltk.tokenize import word_tokenize 
from collections import Counter
nltk.download('punkt')
from konlpy.tag import Okt
okt = Okt()
import re 
from collections import Counter
from PIL import Image
import matplotlib.pylab as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
from matplotlib import pyplot as plt

     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 86 kB 7.6 MB/s 
     |████████████████████████████████| 448 kB 79.7 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import os, sys
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
my_path = '/content/drive/MyDrive/additional_package'

#!pip install --target=$my_path konlpy

# 다운 받은 패키지 바로 import할 수 있게 하기
sys.path.append('/content/drive/MyDrive/additional_package')

os.chdir('/content/drive/MyDrive/Colab Notebooks')

  Using cached konlpy-0.5.2-py2.py3-none-any.whl (19.4 MB)
     |████████████████████████████████| 6.4 MB 7.4 MB/s 
  Using cached beautifulsoup4-4.6.0-py3-none-any.whl (86 kB)
  Using cached JPype1-1.3.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (448 kB)
     |████████████████████████████████| 15.7 MB 28.0 MB/s 
     |████████████████████████████████| 65 kB 5.0 MB/s 
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
     |████████████████████████████████| 62 kB 1.1 MB/s 
     |████████████████████████████████| 61 kB 10.2 MB/s 
     |████████████████████████████████| 149 kB 24.5 MB/s 
     |████████████████████████████████| 138 kB 45.7 MB/s 
     |████████████████████████████████| 146 kB 71.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompati

In [ ]:
import pandas as pd
from pandas import DataFrame  as df
import konlpy
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### **통코드**

In [ ]:
# perfume_df= pd.read_csv("perfume_all.csv", encoding='cp949')
perfume_df= pd.read_csv('/content/drive/Shareddrives/KMS/데이터통합/perfume_all.csv')
perfume_df.head(5)

# product= 이름 종류 모음
product=perfume_df.Name.value_counts().index

# 여러 행의 리뷰를 한 리스트로 저장
def review(i): 
  return str(perfume_df[perfume_df['Name']==product[i]].Review.values)

################################################################################################################

#리뷰를 제품별로 하나의 값에 모은 새로운 데이터 프레임 만들기
pf_new_df= df(columns=["Name","Price","document"])

# 가격이 같은 제품들 때문에 아래와 같이 할당

for i in range(len(product)):
  # 각 제품의 가격을 집합으로 중복을 제거하고 가져와서 1대1 대응이 되도록 집합을 다시 리스트로 가져와 전처리
  price= list(set(perfume_df.Price[perfume_df.Name==product[i]]))[0]
  
  # 각 행마다 제품, 가격, 리뷰 가져오기
  pf_new_df.loc[i]=(product[i],price,review(i))


################################################################################################################

# # 향수 종류 라벨링
pf_type=["EDC", "EDT", "EDP", "퍼퓸"]
Type=[]
for pf in pf_new_df.Name:
  for types in pf_type:
    if types in pf:
      Type.append(types)

# pf_new_df["Type"]= Type
# pf_new_df= pf_new_df[["Name","Type","Price","document"]]
# pf_new_df= pf_new_df.reindex(columns=["Name","Type","Price","document"])
pf_new_df
################################################################################################################

import re  # 정규식 표현 모듈 re

# 한글과 띄어쓰기 외 다른 요소 제거 함수 spec_words 생성
def spec_word(s): 
  hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
  result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거 
  return(result)

# 모든 향수에 적용
for i in range(len(product)):
  pf_new_df.document[i]=spec_word(pf_new_df.document[i])


from konlpy.tag import Okt
import re 
from nltk.tokenize import word_tokenize 
from collections import Counter

################################################################################################################

k=0
corpus=[] # 텍스트 전처리가 완료된 리뷰 데이터 모음

################################################################################################################
for document in pf_new_df.document:

  okt = Okt()

  # 형태소
  morph=okt.morphs(document)

  morph= spec_word(str(morph))

  #품사 태깅
  posing= okt.pos(morph)


  ################################################################################################################

  # 키워드 추출 시 의미가 없이 많이 사용되는 동사, 조사, 접미사, 접속사 제거
  
  pos=[]
  for item in posing:
    if "Verb" not in item and "Josa" not in item and "Suffix" not in item and "Conjunction" not in item:
      pos.append(item)
 

  ################################################################################################################

  # 영어와 특수문자 제거하기
  # 정규 표현식 함수 정의
  pos=str(pos)

  def spec_word(s): 
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거 
    return(result)

  pos= spec_word(pos)
  pos= pos.split(" ")
  # 형태소에서 한 글자 키워드는 제외하기
  pos= [a for a in pos if len(a) > 1]  
  pos= spec_word(str(pos))


  # 자음 모음  제거해주기
  repl =''
  pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'
  result = re.sub(pattern= pattern, repl=repl, string=str(pos))
  pos=result.split(' ')
  

  ################################################################################################################
  #불용어 제거하기
  #통상적으로 의미를 갖지 않는 불용어들의 사전을 RANKS NL에서 받아서 활용
  stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()

  #향수 리뷰에 자주 등장하지만 의미가 없는 단어들을 추가
  perfume_stopwords=["향수", "제품", "향기", "취향","냄새","너무","같아요","사이즈","생각","느낌","입니다","그냥","같은"] 

  stopword= [i[0] for i in stopwords] +perfume_stopwords

  ################################################################################################################

  # 불용어 제거하기
  
  pos= word_tokenize(spec_word(str(pos)))

  def remove_stopwords(sentence): # 불용어 제거 함수 remove_stopwords
    word_tokens = sentence

    result = [word for word in word_tokens if word not in stopword]
    return result

  keywords=remove_stopwords(pos)


  
  ################################################################################################################

  # 키워드 최다빈도 단어 도출

  print("향수",product[k],"의 핵심 키워드:",Counter(keywords).most_common(20))
  k+=1

  # TF-IDF에 사용할 정제된 리뷰 데이터
  
  corpus.append(spec_word(str(keywords)))

향수 조말론 블랙베리 앤 베이 오드코롱, 30ml 의 핵심 키워드: [('명품', 370), ('선물', 362), ('구매', 258), ('좋아요', 231), ('포장', 210), ('배송', 209), ('조말', 181), ('지속', 139), ('사용', 134), ('블랙베리', 108), ('가격', 87), ('백화점', 87), ('좋은', 84), ('샘플', 78), ('쇼핑', 77), ('구입', 77), ('주문', 71), ('상품', 71), ('많이', 70), ('정말', 67)]
향수 클린 웜코튼 EDP 30ml (N) 기프트세트 의 핵심 키워드: [('좋아요', 300), ('선물', 282), ('지속', 273), ('구매', 243), ('비누', 221), ('좋은', 188), ('코튼', 176), ('사용', 143), ('뿌리', 135), ('클린', 132), ('오래', 100), ('포장', 100), ('정말', 99), ('진짜', 98), ('섬유', 80), ('좋아서', 79), ('처음', 78), ('추천', 78), ('친구', 72), ('좋습니다', 69)]
향수 마크제이콥스 스플래시 레인 오드뚜왈렛 100ml 의 핵심 키워드: [('구매', 347), ('여름', 320), ('지속', 276), ('좋아요', 269), ('사용', 204), ('레인', 197), ('뿌리', 167), ('좋은', 160), ('많이', 123), ('잔향', 123), ('정말', 122), ('시원한', 115), ('중성', 115), ('선물', 111), ('진짜', 111), ('이향', 107), ('아주', 107), ('오이', 106), ('추천', 100), ('처음', 95)]
향수 랑방 에끌라드 아르페쥬 EDP 30ml 의 핵심 키워드: [('좋아요', 317), ('지속', 214), ('구매', 210), ('선물', 196), ('랑방', 140), ('사용', 

## TF-IDF 

불용어 제거를 통해서도 제거되지 않은 잦게 출현하는 단어들에게 패널티를 주기 위해 TF-IDF를 실행한다.

In [ ]:
#함수 불러오기
from math import log
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# docs= corpus
keyword = list(set(w for doc in corpus for w in doc.split()))

In [ ]:
#tf-idf 수행
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(corpus)
print(tfidf_matrix.shape)

(279, 25790)


In [ ]:
tfidf_ = pd.DataFrame(tfidf_matrix.toarray(),columns = keyword)
tfidf_

,긍정,드립,뽀득뽀득,함부로,체내,시퉌,어이없아서,감초,따땃,터너,개운하다,시간대,딱이라고,원할,웃음,안되죠,놀랬고,추적추적,군용,일과,문제기,향꽃향,트향를,주도,싫다고하는,쪼꿈,비슷하네여,존재,좋아하는데이건,햣긋,까다롭지,그렇게,아닌것처럼,섬세해서,몸므,오데뚜왈렛,완포,귀여우면서도,반했어,이상해서,...,보성,있을지도,구간,귀여워여,완전히,엊그제,이상정,워터파크,뭐살,아니지요,아깝네,나딘,닥위입,재촉,비싸죠,같아대,없을,찝찝해여,있는줄,기사,듬향,같고여,분산,저렴하고,휴지,진했지만,제시,엠보,피어싱,시원하지만,편하겠다,진함이,정이,아프더라구요,아니었거든,절도,진품,으른향,보아,미되
0,0.0,0.000000,0.000000,0.067046,0.003868,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.008019,0.00000,0.0,0.003868,0.0,0.0,0.0,0.001517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.004376,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.003868,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002337,0.0,0.0,0.0,0.0,0.002514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003868
1,0.0,0.014648,0.000000,0.067123,0.000000,0.0,0.0,0.0,0.0,0.0,0.004723,0.000000,0.000000,0.00398,0.0,0.000000,0.0,0.0,0.0,0.003831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.005523,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.000000,0.005240,0.057710,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.003551,0.001975,0.00000,0.0,0.000000,0.0,0.0,0.0,0.004485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.001437,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.003551,0.000000,0.0,0.0,0.003811,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004606,0.0,0.0,0.0,0.0,0.002477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.003139,0.068246,0.000000,0.0,0.0,0.0,0.0,0.0,0.009940,0.000000,0.003551,0.00000,0.0,0.000000,0.0,0.0,0.0,0.002687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.005166,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.000000,0.044098,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.006373,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012297,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007430,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
275,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
276,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
277,0.0,0.000000

In [ ]:
data = pd.concat([pf_new_df,tfidf_],axis=1)

### **키워드 검색모형**

In [ ]:
# 키워드 검색 모형

key = input("찾고싶은 향수의 키워드를 입력하세요: ")
key= okt.morphs(key)[0]
data1 = data.sort_values(by=[key],axis=0,ascending=False)
# 키워드 검색 시 해당 단어를 기준으로 내림차순 정렬
data2 = data1.iloc[:,[0,1]]
# data2 = pd.merge(data2,pf_senti, how='outer',on='Name')
# data2 = data2.drop(index=[0], axis=0)
data2.head(5)
# 검색한 이용지에게 향수이름/가격/리뷰 까지 다 보여주는 것이 좋을 것 같습니다.
# 지금 리뷰를 명사 추출한 것으로 추가했는데, 본래 리뷰 텍스트로 추가하면 더 좋을 것 같습니다.

찾고싶은 향수의 키워드를 입력하세요: 대학생


,Name,Price
140,"페라가모 인칸토 참 EDT, 30ml","14,900"
184,필로소피 퓨어그레이스 프레그런스EDT 60ml,"55,200"
190,토스 EDT 30ml,"43,000"
189,제니퍼로페즈 엔듀어링 글로우 EDP 30ml,"55,000"
188,플라워바이겐조 포피 부케 EDP 플로럴 쿠션(향수쿠션),"24,000"


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(data.index, index=data['Name']).drop_duplicates()
print(indices.head())

Name
조말론 블랙베리 앤 베이 오드코롱, 30ml      0
클린 웜코튼 EDP 30ml (N) 기프트세트     1
마크제이콥스 스플래시 레인 오드뚜왈렛 100ml    2
랑방 에끌라드 아르페쥬 EDP 30ml         3
[NEW] 클린 웜코튼 EDP(N) 60ml      4
dtype: int64


### **선택한 향수와 가장 유사향수 추천**

In [ ]:
def get_recommendations(Name, cosine_sim=cosine_sim):
    # 선택한 향수의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 향수를 가지고 연산할 수 있습니다.
    idx = indices[Name]

    # 모든 향수에 대해서 해당 향수와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 향수들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 5개의 향수를 받아옵니다.
    sim_scores = sim_scores[1:6]

    # 가장 유사한 5개의 향수의 인덱스를 받아옵니다.
    perfume_indices = [i[0] for i in sim_scores]

    # 가장 유사한 5개의 향수의 제목을 리턴합니다.
    return data['Name'].iloc[perfume_indices]

In [ ]:
# 향수제목 id를 매핑할 dictionary를 생성해줍니다. 
perfume2id = {}
for i, c in enumerate(pf_new_df['Name']): perfume2id[i] = c

# id와 향수이름을 매핑할 dictionary를 생성해줍니다. 
id2perfume= {}
for i, c in perfume2id.items(): id2perfume[c] = i

In [ ]:
# 향수 인덱스
idx = id2perfume['페라가모 인칸토 참 EDT 30ml']
# 자기 자신을 제외한 향수들의 유사도 및 인덱스를 추출  
sim_scores = [(i, c) for i, c in enumerate(cosine_sim[idx]) if i != idx]
# 유사도가 높은 순서대로 정렬
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
# 상위 5개의 인덱스와 유사도를 추출 
sim_scores[0:5]

[(52, 0.8331139101651115),
 (3, 0.6782049229687944),
 (45, 0.6501262161477502),
 (140, 0.6469601585241269),
 (24, 0.6462703566941008)]

In [ ]:
key = input("찾고싶은 향수의 키워드를 입력하세요: ")
get_recommendations(str(key))

찾고싶은 향수의 키워드를 입력하세요: 대학생


KeyError: ignored

###감성분석

In [ ]:
perfume_df= pd.read_csv('/content/drive/Shareddrives/KMS/최종코드/senti_pf1.csv', encoding='cp949')

# product= 이름 종류 모음
product=perfume_df.Name.value_counts().index

# 여러 행의 리뷰를 한 리스트로 저장
def review(i): 
  return str(perfume_df[perfume_df['Name']==product[i]].review.values)

################################################################################################################

#리뷰를 제품별로 하나의 값에 모은 새로운 데이터 프레임 만들기
pf_new_df= df(columns=["Name","Price","document"])

# 가격이 같은 제품들 때문에 아래와 같이 할당

for i in range(len(product)):
  # 각 제품의 가격을 집합으로 중복을 제거하고 가져와서 1대1 대응이 되도록 집합을 다시 리스트로 가져와 전처리
  price= list(set(perfume_df.price[perfume_df.Name==product[i]]))[0]
  
  # 각 행마다 제품, 가격, 리뷰 가져오기
  pf_new_df.loc[i]=(product[i],price,review(i))


################################################################################################################

import re  # 정규식 표현 모듈 re

# 한글과 띄어쓰기 외 다른 요소 제거 함수 spec_words 생성
def spec_word(s): 
  hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
  result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거 
  return(result)

# 모든 향수에 적용
for i in range(len(product)):
  pf_new_df.document[i]=spec_word(pf_new_df.document[i])


from konlpy.tag import Okt
import re 
from nltk.tokenize import word_tokenize 
from collections import Counter

################################################################################################################

k=0
corpus1=[] # 텍스트 전처리가 완료된 리뷰 데이터 모음

################################################################################################################
for document in pf_new_df.document:

  okt = Okt()

  # 형태소
  morph=okt.morphs(document)

  morph= spec_word(str(morph))

  #품사 태깅
  posing= okt.pos(morph)


  ################################################################################################################

  # 키워드 추출 시 의미가 없이 많이 사용되는 동사, 조사, 접미사, 접속사 제거
  
  pos=[]
  for item in posing:
    if "Verb" not in item and "Josa" not in item and "Suffix" not in item and "Conjunction" not in item:
      pos.append(item)
 

  ################################################################################################################

  # 영어와 특수문자 제거하기
  # 정규 표현식 함수 정의
  pos=str(pos)

  def spec_word(s): 
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거 
    return(result)

  pos= spec_word(pos)
  pos= pos.split(" ")
  # 형태소에서 한 글자 키워드는 제외하기
  pos= [a for a in pos if len(a) > 1]  
  pos= spec_word(str(pos))


  # 자음 모음  제거해주기
  repl =''
  pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'
  result = re.sub(pattern= pattern, repl=repl, string=str(pos))
  pos=result.split(' ')
  

  ###############################################################################################################
  #불용어 제거하기
  #통상적으로 의미를 갖지 않는 불용어들의 사전을 RANKS NL에서 받아서 활용
  stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()

  #향수 리뷰에 자주 등장하지만 의미가 없는 단어들을 추가
  perfume_stopwords=["향수", "제품", "향기", "취향","냄새","너무","같아요","사이즈","생각","느낌","입니다","그냥","같은"] 

  stopword= [i[0] for i in stopwords] +perfume_stopwords

  ################################################################################################################

  # 불용어 제거하기
  
  pos= word_tokenize(spec_word(str(pos)))

  def remove_stopwords(sentence): # 불용어 제거 함수 remove_stopwords
    word_tokens = sentence

    result = [word for word in word_tokens if word not in stopword]
    return result

  keywords=remove_stopwords(pos)


  
  ################################################################################################################

  # 키워드 최다빈도 단어 도출

  print("향수",product[k],"의 핵심 키워드:",Counter(keywords).most_common(20))
  globals()['corpus1_{}'.format(k)]=[]
  globals()['corpus1_{}'.format(k)].append(spec_word(str(keywords)))
  k+=1

  # TF-IDF에 사용할 정제된 리뷰 데이터
  
  corpus1.append(spec_word(str(keywords)))

향수 조말론 블랙베리 앤 베이 오드코롱, 30ml 의 핵심 키워드: [('달콤한', 6), ('조말', 5), ('가격', 5), ('블랙베리', 5), ('이나', 4), ('정말', 4), ('정품', 3), ('내외', 3), ('베이', 3), ('포장', 3), ('상자', 3), ('그런', 2), ('카드', 2), ('할인', 2), ('병행수입', 2), ('만원', 2), ('본인', 2), ('단점', 2), ('있는데', 2), ('많은', 2)]
향수 필로소피 퓨어그레이스 프레그런스EDT 60ml 의 핵심 키워드: [('선물', 101), ('사용', 79), ('좋아요', 77), ('가격', 71), ('구매', 68), ('오래', 55), ('지속', 44), ('불가리', 43), ('배송', 41), ('구입', 39), ('많이', 36), ('알콜', 33), ('정품', 31), ('뿌리', 30), ('남편', 30), ('남자', 28), ('좋은', 28), ('정말', 27), ('항상', 27), ('정도', 26)]
향수 제니퍼로페즈 엔듀어링 글로우 EDP 30ml 의 핵심 키워드: [('좋아요', 77), ('구매', 70), ('버버리', 62), ('가격', 45), ('사용', 41), ('오래', 41), ('정품', 38), ('배송', 37), ('지속', 32), ('알콜', 28), ('쿠팡', 27), ('겨울', 23), ('뿌리', 22), ('금방', 22), ('많이', 19), ('항상', 19), ('상품', 19), ('우먼', 19), ('구입', 19), ('역시', 18)]
향수 플라워바이겐조 포피 부케 EDP 플로럴 쿠션(향수쿠션) 의 핵심 키워드: [('좋아요', 89), ('배송', 56), ('구매', 52), ('선물', 48), ('가격', 42), ('오래', 40), ('잔향', 26), ('지속', 24), ('좋고', 23), ('주문', 20), ('사용'

In [ ]:
#함수 불러오기
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#tf-idf 수행

tfidf = TfidfVectorizer(stop_words='english')
for i in range(len(product)):
  globals()['corpus1_{}'.format(i)]
  globals()['tfidf_matrix_{}'.format(i)] = tfidf.fit_transform(globals()['corpus1_{}'.format(i)])

###데이터불러오기

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
def spec_word(s): 
  hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
  result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
  return(result)

def rating_to_label(rating):
    if rating > 2:
        return 1
    else:
        return 0
        
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 정규 표현식 처리
    result = hangul.sub('', text)
    okt = Okt()  # 형태소 추출
    nouns = okt.nouns(result)
    nouns = [x for x in nouns if len(x) > 1]  # 한글자 키워드 제거
    nouns = [x for x in nouns if x not in stopwords]  # 불용어 제거
    return nouns

###키워드 추출

In [ ]:
df= pd.read_csv("/content/drive/Shareddrives/KMS/최종코드/senti_pf1.csv", encoding='cp949')
for i in range(len(product)):
  globals()['df_{}'.format(i)]=df[df.Name==product[i]]
  globals()['df_{}'.format(i)].reset_index(inplace=True)
  globals()['df_{}'.format(i)].drop('index', axis=1, inplace=True)

  globals()['df_{}'.format(i)]['review'][i]=spec_word(globals()['df_{}'.format(i)]['review'][i])
  globals()['df_{}'.format(i)]['y'] =  globals()['df_{}'.format(i)]['Rating'].apply(lambda x: rating_to_label(x))
  
  vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
  bow_vect = vect.fit_transform(globals()['df_{}'.format(i)]['review'].tolist())

  tfidf_vectorizer = TfidfTransformer()
  tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)
  x = tf_idf_vect
  y = globals()['df_{}'.format(i)]['y']
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=1)


  # fit in training set
  lr = LogisticRegression(random_state = 0)
  lr.fit(x_train, y_train)

  # predict in test set
  y_pred = lr.predict(x_test)

  coef_pos_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = True)
  coef_neg_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = False)

  invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}
  
  pos_keyword=[]
  pos_coef=[]

  for coef in coef_pos_index[:10]:
      a = '# '+invert_index_vectorizer[coef[1]]
      pos_keyword.append(a)
      #pos_keyword.append('# '&str(invert_index_vectorizer[coef[1]]))
  neg_keyword=[]
  for coef in coef_neg_index[:10]:
      b = '# '+invert_index_vectorizer[coef[1]]
      neg_keyword.append(b)
      #neg_keyword.append(str(invert_index_vectorizer[coef[1]]))
  globals()['pf_senti{}'.format(i)] = pd.DataFrame({'Name' : product[i], 'positive' : [pos_keyword], 'negative' : [neg_keyword]})

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/py

In [ ]:
pf_senti = pd.concat([pf_senti0,pf_senti1,pf_senti2,pf_senti3,pf_senti4])
pf_senti.reset_index(inplace=True)
pf_senti.drop('index', axis=1, inplace=True)
pf_senti

,Name,positive,negative
0,"조말론 블랙베리 앤 베이 오드코롱, 30ml","[# 배송, # 포장, # 가격, # 사용, # 친구, # 구입, # 아주, # 조...","[# 가짜, # 가품, # 그냥, # 기존, # 문의, # 뚜껑, # 답변, # 실..."
1,필로소피 퓨어그레이스 프레그런스EDT 60ml,"[# 선물, # 남편, # 항상, # 가격, # 배송, # 아주, # 향수, # 한...","[# 알콜, # 냄새, # 그냥, # 이건, # 금방, # 별로, # 짝퉁, # 가..."
2,제니퍼로페즈 엔듀어링 글로우 EDP 30ml,"[# 선물, # 추억, # 지속, # 겨울, # 구매, # 우먼, # 주문, # 포...","[# 짝퉁, # 이건, # 실망, # 금방, # 진품, # 알콜, # 반품, # 냄..."
3,플라워바이겐조 포피 부케 EDP 플로럴 쿠션(향수쿠션),"[# 선물, # 향수, # 가격, # 항상, # 아주, # 여성, # 베르사체, #...","[# 반품, # 전혀, # 정품, # 알콜, # 미국, # 패키지, # 실망, # ..."
4,모스키노 토이2 EDP 50ml 미니어쳐 증정기획,"[# 선물, # 지속, # 가격, # 구매, # 여름, # 잔향, # 상큼, # 남...","[# 백화점, # 사용, # 그냥, # 여자, # 가품, # 금방, # 개월, # ..."


# 최종

In [ ]:
# 키워드 검색 모형

key = input("찾고싶은 향수의 키워드를 입력하세요: ")
key= okt.morphs(key)[0]
data1 = data.sort_values(by=[key],axis=0,ascending=False)
# 키워드 검색 시 해당 단어를 기준으로 내림차순 정렬
data2 = data1.iloc[:,[0,1]]
data2 = pd.merge(data2,pf_senti, how='outer',on='Name')
data2 = data2.drop(index=[0], axis=0)
data2.head(5)
# 검색한 이용지에게 향수이름/가격/리뷰 까지 다 보여주는 것이 좋을 것 같습니다.
# 지금 리뷰를 명사 추출한 것으로 추가했는데, 본래 리뷰 텍스트로 추가하면 더 좋을 것 같습니다.

찾고싶은 향수의 키워드를 입력하세요: 데이트


,Name,Price,positive,negative
1,"조말론 블랙베리 앤 베이 오드코롱, 30ml","56,800","[# 배송, # 포장, # 가격, # 사용, # 친구, # 구입, # 아주, # 조...","[# 가짜, # 가품, # 그냥, # 기존, # 문의, # 뚜껑, # 답변, # 실..."
2,필로소피 퓨어그레이스 프레그런스EDT 60ml,"55,200","[# 선물, # 남편, # 항상, # 가격, # 배송, # 아주, # 향수, # 한...","[# 알콜, # 냄새, # 그냥, # 이건, # 금방, # 별로, # 짝퉁, # 가..."
3,토스 EDT 30ml,"43,000",NaN,NaN
4,제니퍼로페즈 엔듀어링 글로우 EDP 30ml,"55,000","[# 선물, # 추억, # 지속, # 겨울, # 구매, # 우먼, # 주문, # 포...","[# 짝퉁, # 이건, # 실망, # 금방, # 진품, # 알콜, # 반품, # 냄..."
5,플라워바이겐조 포피 부케 EDP 플로럴 쿠션(향수쿠션),"24,000","[# 선물, # 향수, # 가격, # 항상, # 아주, # 여성, # 베르사체, #...","[# 반품, # 전혀, # 정품, # 알콜, # 미국, # 패키지, # 실망, # ..."
